# Import libraries

In [1]:
import pandas as pd
from scipy.stats import wilcoxon
import matplotlib.pyplot as plt
from matplotlib import rc
import numpy as np
import utils
import config
import re
import os

In [2]:
# set figure details
# rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
# rc('text', usetex=True)
rc("svg", fonttype="none") 

# Metadata

In [3]:
metadata = pd.read_csv(os.path.join(config.CLEAN_DIR, "metadata.tsv"), sep="\t", index_col=0)
metadata["date_time"] = pd.to_datetime(metadata['Date_Collected'])
metadata

,Patient_No,Hospital,Date_Collected,Time_Collected,Timeline_Weeks,Abx_regular,Abx_anomoly,Abx_timeline.prior..during..after.,abx_day,Consistency,...,expo_cat2,calc_age_mo,currentfeed_bf,currentfeed_f,currentfeed_o,Sequencing_Project,Filename,Extraction,depth,date_time
Sample_ID,,,,,,,,,,,,,,,,,,,,,
magic.0001,101,no,12/10/16,1430,0,regular,NaN,NaN,NaN,NaN,...,Vaginal,0.07,1.0,0.0,magic.0001,Knights_Project_046,magic.0001.S22.001.fa,MagAttract,1544781,2016-12-10
magic.0004,101,no,12/15/16,NaN,1,regular,NaN,NaN,NaN,NaN,...,Vaginal,0.23,1.0,0.0,magic.0004,Knights_Project_046,magic.0004.S34.001.fa,MagAttract,1167038,2016-12-15
magic.0006,104,yes,12/15/16,2335,0,regular,NaN,NaN,NaN,NaN,...,Vaginal,0.03,1.0,1.0,magic.0006,Knights_Project_076_Pool3,magic.0006.S145.001.fa,PowerSoil,1720636,2016-12-15
magic.0007,105,yes,12/15/16,2115,0,regular,NaN,NaN,NaN,NaN,...,Vaginal,0.03,1.0,1.0,magic.0007,Knights_Project_076_Pool3,magic.0007.S155.001.fa,PowerSoil,6825017,2016-12-15
magic.0008,105,yes,12/15/16,1525,0,regular,NaN,NaN,NaN,NaN,...,Vaginal,0.03,1.0,1.0,magic.0008,Knights_Project_055,X8.S125.001.fa,PowerSoil,2928044,2016-12-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
magic.6595,588,NaN,8/22/21,1702,96,regular,NaN,NaN,NaN,NaN,...,Vaginal,24.11,NaN,NaN,magic.6595,Knights_Project_077_Pool3,magic.6595.S253.001.fa,PowerSoilPro,1562968,2021-08-22
magic.6597,595,NaN,9/6/21,NaN,96,regular,NaN,NaN,NaN,formed,...,Vaginal,24.11,NaN,NaN,magic.6597,Knights_Project_077_Pool3,magic.6597.S265.001.fa,PowerSoilPro,2113880,2021-09-06
magic.6598,585,NaN,8/23/21,725,96,regular,NaN,NaN,NaN,formed,...,C-section with labor,24.34,NaN,NaN,magic.6598,Knights_Project_077_Pool3,magic.6598.S277.001.fa,PowerSoilPro,1650388,2021-08-23


In [4]:
df = pd.read_csv("../data/raw/species_rarefied.txt", sep="\t")
df = df.T

In [5]:
smoothed = df + 1
relative_abundances = smoothed.div(smoothed.sum(axis=1), axis=0)

In [6]:
from skbio.stats.composition import clr

In [7]:
transformed = pd.DataFrame(clr(relative_abundances), index=relative_abundances.index, columns=relative_abundances.columns)
transformed

,k__Bacteria;p__Firmicutes;c__Bacilli;o__Lactobacillales;f__Enterococcaceae;g__Enterococcus;s__Enterococcus_faecalis,k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Enterobacterales;f__Enterobacteriaceae;g__Klebsiella;s__Klebsiella_pneumoniae,k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Enterobacterales;f__Enterobacteriaceae;g__Escherichia;s__Escherichia_coli,k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Enterobacterales;f__Enterobacteriaceae;g__Escherichia;s__Escherichia_coli_C,k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Enterobacterales;f__Enterobacteriaceae;g__Escherichia;s__Escherichia_coli_D,k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Enterobacterales;f__Enterobacteriaceae;g__Escherichia;s__Escherichia_flexneri,k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Enterobacterales;f__Enterobacteriaceae;g__Leclercia;s__Leclercia_sp002902985,k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Enterobacterales;f__Enterobacteriaceae;g__Escherichia;s__Escherichia_sp005843885,k__Bacteria;p__Firmicutes;c__Bacilli;o__Staphylococcales;f__Staphylococcaceae;g__Staphylococcus;s__Staphylococcus_epidermidis,k__Bacteria;p__Firmicutes;c__Bacilli;o__Lactobacillales;f__Streptococcaceae;g__Streptococcus;s__Streptococcus_pseudopneumoniae_O,...,k__Bacteria;p__Patescibacteria;c__CPR2;o__CG2-30-33-46;f__CG2-30-33-46;g__CG2-30-33-46;s__CG2-30-33-46_sp002792735,k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Pseudomonadales;f__Cellvibrionaceae;g__Agarilytica;s__Agarilytica_rhodophyticola,k__Bacteria;p__Actinobacteriota;c__Actinomycetia;o__Mycobacteriales;f__Pseudonocardiaceae;g__Pseudonocardia;s__Pseudonocardia_sp001294645,k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Burkholderiales;f__Gallionellaceae;g__Nitrotoga;s__Nitrotoga_sp002470125,k__Bacteria;p__Firmicutes_A;c__Clostridia;o__Lachnospirales;f__Lachnospiraceae;g__UBA935;s__UBA935_sp002293525,k__Bacteria;p__Firmicutes;c__Bacilli;o__RFN20;f__CAG-288;g__UBA7642;s__UBA7642_sp900313465,k__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Flavobacteriales;f__Flavobacteriaceae;g__Flavobacterium;s__Flavobacterium_aquidurense_B,k__Bacteria;p__Verrucomicrobiota;c__Verrucomicrobiae;o__Pedosphaerales;f__UBA9464;g__SXXZ01;s__SXXZ01_sp005789375,k__Bacteria;p__Firmicutes;c__Bacilli;o__Bacillales;f__Amphibacillaceae;g__Virgibacillus;s__Virgibacillus_sp000480335,k__Bacteria;p__Firmicutes_A;c__Clostridia;o__Lachnospirales;f__Lachnospiraceae;g__UBA1066;s__UBA1066_sp900319795
magic.0007,4.527748,5.298031,10.445931,6.695183,11.913423,7.874668,0.962921,4.964175,1.473746,-0.135691,...,-0.135691,-0.135691,-0.135691,-0.135691,-0.135691,-0.135691,-0.135691,-0.135691,-0.135691,-0.135691
magic.0008,0.606004,4.889591,9.983848,6.200715,11.496483,6.501783,0.606004,4.055992,0.606004,-0.087143,...,-0.087143,-0.087143,-0.087143,-0.087143,-0.087143,-0.087143,-0.087143,-0.087143,-0.087143,-0.087143
magic.0010,8.071337,4.905097,10.083784,7.582250,9.785048,10.175271,-0.098849,6.566835,3.397658,-0.098849,...,-0.098849,-0.098849,-0.098849,-0.098849,-0.098849,-0.098849,-0.098849,-0.098849,-0.098849,-0.098849
magic.0012,3.914751,6.104414,11.128644,7.163021,12.652704,7.510437,1.223508,5.007698,5.805921,3.169418,...,-0.162786,-0.162786,-0.162786,-0.162786,-0.162786,-0.162786,-0.162786,-0.162786,-0.162786,-0.162786
magic.0014,10.589866,9.506439,13.428556,8.476886,10.327738,8.506733,0.579589,6.383217,6.528929,2.525499,...,-0.113558,-0.113558,-0.113558,-0.113558,-0.113558,-0.113558,-0.113558,-0.113558,-0.113558,-0.113558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
magic.6595,1.646243,5.313005,5.215776,1.646243,3.741189,2.457173,0.953096,0.953096,0.665414,1.964697,...,-0.433198,-0.433198,-0.433198,-0.433198,-0.433198,-0.433198,-0.433198,-0.433198,-0.433198,-0.433198
magic.6597,1.554523,1.688054,3.074348,1.911198,3.719486,2.247670,-0.391388,0.301760,1.688054,0.301760,...,-0.391388,-0.391388,-0.391388,-0.391388,-0.391388,-0.39

In [11]:
metadata = metadata.loc[transformed.index]
metadata

,Patient_No,Hospital,Date_Collected,Time_Collected,Timeline_Weeks,Abx_regular,Abx_anomoly,Abx_timeline.prior..during..after.,abx_day,Consistency,...,expo_cat2,calc_age_mo,currentfeed_bf,currentfeed_f,currentfeed_o,Sequencing_Project,Filename,Extraction,depth,date_time
magic.0007,105,yes,12/15/16,2115,0,regular,NaN,NaN,NaN,NaN,...,Vaginal,0.03,1.0,1.0,magic.0007,Knights_Project_076_Pool3,magic.0007.S155.001.fa,PowerSoil,6825017,2016-12-15
magic.0008,105,yes,12/15/16,1525,0,regular,NaN,NaN,NaN,NaN,...,Vaginal,0.03,1.0,1.0,magic.0008,Knights_Project_055,X8.S125.001.fa,PowerSoil,2928044,2016-12-15
magic.0010,101,no,12/10/16,2230,0,regular,NaN,NaN,NaN,NaN,...,Vaginal,0.07,1.0,0.0,magic.0010,Knights_Project_076_Pool3,magic.0010.S167.001.fa,PowerSoil,7251673,2016-12-10
magic.0012,105,no,12/22/16,950,1,regular,NaN,NaN,NaN,soft,...,Vaginal,0.26,1.0,1.0,magic.0012,Knights_Project_076_Pool3,magic.0012.S177.001.fa,PowerSoil,8106772,2016-12-22
magic.0014,106,no,12/22/16,250,1,regular,NaN,NaN,NaN,watery,...,Vaginal,0.23,1.0,0.0,magic.0014,Knights_Project_068_Pool1,magic.0014.S1.001.fa,PowerSoil,15346610,2016-12-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
magic.6595,588,NaN,8/22/21,1702,96,regular,NaN,NaN,NaN,NaN,...,Vaginal,24.11,NaN,NaN,magic.6595,Knights_Project_077_Pool3,magic.6595.S253.001.fa,PowerSoilPro,1562968,2021-08-22
magic.6597,595,NaN,9/6/21,NaN,96,regular,NaN,NaN,NaN,formed,...,Vaginal,24.11,NaN,NaN,magic.6597,Knights_Project_077_Pool3,magic.6597.S265.001.fa,PowerSoilPro,2113880,2021-09-06
magic.6598,585,NaN,8/23/21,725,96,regular,NaN,NaN,NaN,formed,...,C-section with labor,24.34,NaN,NaN,magic.6598,Knights_Project_077_Pool3,magic.6598.S277.001.fa,PowerSoilPro,1650388,2021-08-23
magic.6599,573,NaN,7/21/21,845,96,regular,NaN,NaN,NaN,soft,...,C-section with labor,24.08,NaN,NaN,magic.6599,Knights_Project_077_Pool3,magic.6599.S288.001.fa,PowerSoilPro,1519641,2021-07-21


In [12]:
ptr = pd.read_csv(os.path.join(config.CLEAN_DIR, "ptr.csv"), index_col=0)
ptr = ptr[metadata.index]
ptr

,magic.0007,magic.0008,magic.0010,magic.0012,magic.0014,magic.0017,magic.0019,magic.0022,magic.0023,magic.0024,...,magic.6590,magic.6591,magic.6592,magic.6593,magic.6594,magic.6595,magic.6597,magic.6598,magic.6599,magic.6600
gtdb_taxonomy,,,,,,,,,,,,,,,,,,,,,
d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactobacillales;f__Streptococcaceae;g__Streptococcus;s__Streptococcus thermophilus,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.890374,0.000000,0.000000,0.000000,0.000000,0.000000,0.398078,0.000000,0.443034
d__Bacteria;p__Firmicutes;c__Clostridia;o__Oscillospirales;f__Oscillospiraceae;g__Intestinimonas;s__Intestinimonas butyriciproducens,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.181598,0.000000,0.000000,0.000000,0.153008,0.164560,0.000000,0.181179,0.000000,0.364100
d__Bacteria;p__Verrucomicrobia;c__Verrucomicrobiae;o__Verrucomicrobiales;f__Akkermansiaceae;g__Akkermansia;s__Akkermansia muciniphila,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.531152,0.000000,0.000000,0.000000,0.000000,0.385291,0.339425
d__Bacteria;p__Firmicutes;c__Clostridia;o__Oscillospirales;f__Ruminococcaceae;g__Ruminiclostridium_E;s__Ruminiclostridium_E siraeum,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.366896,0.000000,0.000000,0.000000,0.000000,0.000000
d__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Bifidobacteriaceae;g__Bifidobacterium;s__Bifidobacterium longum,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.573777,0.0,...,0.000000,0.648231,0.639207,0.000000,0.279416,0.547436,0.706284,0.624992,0.793504,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
d__Bacteria;p__Firmicutes;c__Clostridia;o__Lachnospirales;f__Lachnospiraceae;g__Blautia;s__,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
d__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Actinomycetaceae;g__Actinomyces_B;s__,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
d__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Betaproteobacteriales;f__Burkholderiaceae;g__Diaphorobacter;s__Diaphorobacter ebreus,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [13]:
from scipy.stats import ranksums


for week in metadata["Timeline_Weeks"].unique():
    if str(week) in ["nan", "na"]: continue
    no_abx = metadata[(metadata["expo_cat1"] == "No antibiotics") & (metadata["Timeline_Weeks"] == week)].index
    abx = metadata[(metadata["expo_cat1"] != "No antibiotics") & (metadata["Timeline_Weeks"] == week)].index
#     no_abx = metadata[(metadata["expo_cat1"] == "No antibiotics")].index
#     abx = metadata[(metadata["expo_cat1"] != "No antibiotics")].index
    ptr_noabx = ptr[no_abx]
    ptr_abx = ptr[abx]
    
    wilc_info = []
    
    for bug in ptr.index:
        if len(ptr_noabx.loc[bug].shape) == 2: continue
        a, b = ptr_noabx.loc[bug], ptr_abx.loc[bug]
        info = ranksums(a, b)
        stat, p = info.statistic, info.pvalue
        wilc_info.append({"p" : p, "stat" : stat, "bug" : bug})
        
    wilc_info_df = pd.DataFrame(wilc_info)
    wilc_info_df = wilc_info_df.set_index("bug")
    sorted_ = wilc_info_df.sort_values("p")
    print("week", week)
    display(sorted_[sorted_["p"] < 0.05])

week 0


,p,stat
bug,,


week 1


,p,stat
bug,,
d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactobacillales;f__Enterococcaceae;g__Enterococcus;s__Enterococcus faecalis,0.000027,-4.200077
d__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Bifidobacteriaceae;g__Bifidobacterium;s__Bifidobacterium longum,0.000040,4.106039


week 2


...

,p,stat
bug,,


week 84


,p,stat
bug,,


week 96


,p,stat
bug,,
d__Bacteria;p__Firmicutes;c__Clostridia;o__Oscillospirales;f__Oscillospiraceae;g__Intestinimonas;s__Intestinimonas butyriciproducens,0.000509,3.4758


In [74]:
data = pd.concat([metadata, ptr.T], axis=1)
data

,Patient_No,Hospital,Date_Collected,Time_Collected,Timeline_Weeks,Abx_regular,Abx_anomoly,Abx_timeline.prior..during..after.,abx_day,Consistency,...,d__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Bifidobacteriaceae;g__Bifidobacterium;s__Bifidobacterium vaginale,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactobacillales;f__Streptococcaceae;g__Streptococcus,d__Bacteria;p__Firmicutes;c__Clostridia;o__Lachnospirales;f__Lachnospiraceae;g__Hungatella;s__,d__Bacteria;p__Firmicutes;c__Clostridia;o__Christensenellales;f__Christensenellaceae;g__Christensenella,d__Bacteria;p__Bacteroidetes;c__Bacteroidia;o__Bacteroidales;f__Porphyromonadaceae;g__Porphyromonas;s__Porphyromonas asaccharolytica,d__Bacteria;p__Firmicutes;c__Clostridia;o__Lachnospirales;f__Lachnospiraceae;g__Blautia;s__,d__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Actinomycetaceae;g__Actinomyces_B;s__,d__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Betaproteobacteriales;f__Burkholderiaceae;g__Diaphorobacter;s__Diaphorobacter ebreus,d__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Corynebacteriales;f__Corynebacteriaceae;g__Lawsonella;s__Lawsonella clevelandensis,d__Bacteria;p__Firmicutes;c__Clostridia;o__Lachnospirales;f__Lachnospiraceae;g__Hungatella
magic.0007,105,yes,12/15/16,2115,0,regular,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
magic.0008,105,yes,12/15/16,1525,0,regular,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
magic.0010,101,no,12/10/16,2230,0,regular,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
magic.0012,105,no,12/22/16,950,1,regular,NaN,NaN,NaN,soft,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
magic.0014,106,no,12/22/16,250,1,regular,NaN,NaN,NaN,watery,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
magic.6595,588,NaN,8/22/21,1702,96,regular,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
magic.6597,595,NaN,9/6/21,NaN,96,regular,NaN,NaN,NaN,formed,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
magic.6598,585,NaN,8/23/21,725,96,regular,NaN,NaN,NaN,formed,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
magic.6599,573,NaN,7/21/21,845,96,regular,NaN,NaN,NaN,soft,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [81]:
data["abx"] = data["expo_cat1"].apply(lambda x : "No antibiotics" if x == "No antibiotics" else "Antibiotics")

In [82]:
data["week_age"] = data["calc_age_mo"] * 4

In [83]:
test_data = data[["abx", "d__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Bifidobacteriaceae;g__Bifidobacterium;s__Bifidobacterium longum", "week_age", "Patient_No"]]
test_data = test_data[~test_data["week_age"].isna()]
test_data

,abx,d__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Bifidobacteriaceae;g__Bifidobacterium;s__Bifidobacterium longum,week_age,Patient_No
magic.0007,No antibiotics,0.000000,0.12,105
magic.0008,No antibiotics,0.000000,0.12,105
magic.0010,No antibiotics,0.000000,0.28,101
magic.0012,No antibiotics,0.000000,1.04,105
magic.0014,No antibiotics,0.000000,0.92,106
...,...,...,...,...
magic.6595,No antibiotics,0.547436,96.44,588
magic.6597,No antibiotics,0.706284,96.44,595
magic.6598,Antibiotics,0.624992,97.36,585
magic.6599,Antibiotics,0.793504,96.32,573


In [84]:
test_data.isna().sum()

abx                                                                                                                                      0
d__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Bifidobacteriaceae;g__Bifidobacterium;s__Bifidobacterium longum    0
week_age                                                                                                                                 0
Patient_No                                                                                                                               0
dtype: int64

In [85]:
test_data.to_csv("/Users/daniel/Documents/School/2020-2021/spring/research/magic/test_splinectomeR/data.tsv", sep="\t", index=False)

In [ ]:
# e faecalis p = 0.009

,magic.0007,magic.0008,magic.0010,magic.0012,magic.0014,magic.0017,magic.0019,magic.0022,magic.0023,magic.0024,...,magic.6590,magic.6591,magic.6592,magic.6593,magic.6594,magic.6595,magic.6597,magic.6598,magic.6599,magic.6600
gtdb_taxonomy,,,,,,,,,,,,,,,,,,,,,
d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactobacillales;f__Streptococcaceae;g__Streptococcus;s__Streptococcus thermophilus,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.890374,0.000000,0.000000,0.000000,0.000000,0.000000,0.398078,0.000000,0.443034
d__Bacteria;p__Firmicutes;c__Clostridia;o__Oscillospirales;f__Oscillospiraceae;g__Intestinimonas;s__Intestinimonas butyriciproducens,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.181598,0.000000,0.000000,0.000000,0.153008,0.164560,0.000000,0.181179,0.000000,0.364100
d__Bacteria;p__Verrucomicrobia;c__Verrucomicrobiae;o__Verrucomicrobiales;f__Akkermansiaceae;g__Akkermansia;s__Akkermansia muciniphila,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.531152,0.000000,0.000000,0.000000,0.000000,0.385291,0.339425
d__Bacteria;p__Firmicutes;c__Clostridia;o__Oscillospirales;f__Ruminococcaceae;g__Ruminiclostridium_E;s__Ruminiclostridium_E siraeum,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.366896,0.000000,0.000000,0.000000,0.000000,0.000000
d__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Bifidobacteriaceae;g__Bifidobacterium;s__Bifidobacterium longum,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.573777,0.0,...,0.000000,0.648231,0.639207,0.000000,0.279416,0.547436,0.706284,0.624992,0.793504,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
d__Bacteria;p__Firmicutes;c__Clostridia;o__Lachnospirales;f__Lachnospiraceae;g__Blautia;s__,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
d__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Actinomycetaceae;g__Actinomyces_B;s__,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
d__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Betaproteobacteriales;f__Burkholderiaceae;g__Diaphorobacter;s__Diaphorobacter ebreus,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [96]:
for week in metadata["Timeline_Weeks"].unique():
    week = str(week)
    if week == "na" or week == "nan": continue
    week_metadata = metadata[metadata["Timeline_Weeks"] == week]
    ptr_curr = ptr[week_metadata.index]
    display(ptr_curr)
    break

,magic.0007,magic.0008,magic.0010,magic.0024,magic.0025,magic.0026,magic.0027,magic.0031,magic.0033,magic.0034,...,magic.2203,magic.2205,magic.2522,magic.2562,magic.2580,magic.2863,magic.3478,magic.3692,magic.4872,magic.5088
gtdb_taxonomy,,,,,,,,,,,,,,,,,,,,,
d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactobacillales;f__Streptococcaceae;g__Streptococcus;s__Streptococcus thermophilus,0.0,0,0.0,0.0,0,0,0,0,0.000000,0.0,...,0.0,0,0.0,0.000000,0.0,0.000000,0.0,0.0,0,0
d__Bacteria;p__Firmicutes;c__Clostridia;o__Oscillospirales;f__Oscillospiraceae;g__Intestinimonas;s__Intestinimonas butyriciproducens,0.0,0,0.0,0.0,0,0,0,0,0.000000,0.0,...,0.0,0,0.0,0.000000,0.0,0.000000,0.0,0.0,0,0
d__Bacteria;p__Verrucomicrobia;c__Verrucomicrobiae;o__Verrucomicrobiales;f__Akkermansiaceae;g__Akkermansia;s__Akkermansia muciniphila,0.0,0,0.0,0.0,0,0,0,0,0.000000,0.0,...,0.0,0,0.0,0.000000,0.0,0.000000,0.0,0.0,0,0
d__Bacteria;p__Firmicutes;c__Clostridia;o__Oscillospirales;f__Ruminococcaceae;g__Ruminiclostridium_E;s__Ruminiclostridium_E siraeum,0.0,0,0.0,0.0,0,0,0,0,0.000000,0.0,...,0.0,0,0.0,0.000000,0.0,0.000000,0.0,0.0,0,0
d__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Bifidobacteriaceae;g__Bifidobacterium;s__Bifidobacterium longum,0.0,0,0.0,0.0,0,0,0,0,1.124119,0.0,...,0.0,0,0.0,0.344078,0.0,0.285392,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
d__Bacteria;p__Firmicutes;c__Clostridia;o__Lachnospirales;f__Lachnospiraceae;g__Blautia;s__,0.0,0,0.0,0.0,0,0,0,0,0.000000,0.0,...,0.0,0,0.0,0.000000,0.0,0.000000,0.0,0.0,0,0
d__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Actinomycetaceae;g__Actinomyces_B;s__,0.0,0,0.0,0.0,0,0,0,0,0.000000,0.0,...,0.0,0,0.0,0.000000,0.0,0.000000,0.0,0.0,0,0
d__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Betaproteobacteriales;f__Burkholderiaceae;g__Diaphorobacter;s__Diaphorobacter ebreus,0.0,0,0.0,0.0,0,0,0,0,0.000000,0.0,...,0.0,0,0.0,0.000000,0.0,0.000000,0.0,0.0,0,0


In [18]:
diff_abunds = []

In [ ]:
from scipy.stats import ranksums


for week in metadata["Timeline_Weeks"].unique():
    if str(week) in ["nan", "na"]: continue
        
    no_abx = metadata[(metadata["expo_cat1"] == "No antibiotics") & (metadata["Timeline_Weeks"] == week)].index
    abx = metadata[(metadata["expo_cat1"] != "No antibiotics") & (metadata["Timeline_Weeks"] == week)].index
    
    species_noabx = transformed.T[no_abx]
    species_abx = transformed.T[abx]
    
    wilc_info = []
    
    for bug in transformed.T.index:
        if len(species_noabx.loc[bug].shape) == 2: continue
        a, b = species_noabx.loc[bug], species_abx.loc[bug]
        info = ranksums(a, b)
        stat, p = info.statistic, info.pvalue
        wilc_info.append({"p" : p, "stat" : stat, "bug" : bug})
        
    wilc_info_df = pd.DataFrame(wilc_info)
    wilc_info_df = wilc_info_df.set_index("bug")
    sorted_ = wilc_info_df.sort_values("p")
    print("week", week)
    info = sorted_[sorted_["p"] < 0.05]
    diff_abunds.append(info)
    display(info)
    

In [45]:
for i, diff in enumerate(diff_abunds):
    try:
        df = diff.loc[[ind for ind in diff.index if "s__Enterococcus_faecalis" in ind]]
        display(df)
    except:
        print("not in timepoint", i)
        


,p,stat
bug,,
k__Bacteria;p__Firmicutes;c__Bacilli;o__Lactobacillales;f__Enterococcaceae;g__Enterococcus;s__Enterococcus_faecalis,0.016548,-2.39659


,p,stat
bug,,
k__Bacteria;p__Firmicutes;c__Bacilli;o__Lactobacillales;f__Enterococcaceae;g__Enterococcus;s__Enterococcus_faecalis,3.845044e-07,-5.076473


,p,stat
bug,,
k__Bacteria;p__Firmicutes;c__Bacilli;o__Lactobacillales;f__Enterococcaceae;g__Enterococcus;s__Enterococcus_faecalis,0.000022,-4.243064


,p,stat
bug,,
k__Bacteria;p__Firmicutes;c__Bacilli;o__Lactobacillales;f__Enterococcaceae;g__Enterococcus;s__Enterococcus_faecalis,0.000639,-3.414591


,p,stat
bug,,
k__Bacteria;p__Firmicutes;c__Bacilli;o__Lactobacillales;f__Enterococcaceae;g__Enterococcus;s__Enterococcus_faecalis,0.000001,-4.822717


...

,p,stat
bug,,


,p,stat
bug,,


,p,stat
bug,,


,p,stat
bug,,


,p,stat
bug,,


In [4]:
for subject in metadata["Patient_No"].unique():
    subject_df = metadata[metadata["Patient_No"] == subject].sort_values("date_time")
    display(subject_df)
    input()

,Patient_No,Hospital,Date_Collected,Time_Collected,Timeline_Weeks,Abx_regular,Abx_anomoly,Abx_timeline.prior..during..after.,abx_day,Consistency,...,expo_cat2,calc_age_mo,currentfeed_bf,currentfeed_f,currentfeed_o,Sequencing_Project,Filename,Extraction,depth,date_time
Sample_ID,,,,,,,,,,,,,,,,,,,,,
magic.0001,101,no,12/10/16,1430,0,regular,NaN,NaN,NaN,NaN,...,Vaginal,0.07,1.0,0.0,magic.0001,Knights_Project_046,magic.0001.S22.001.fa,MagAttract,1544781,2016-12-10
magic.0010,101,no,12/10/16,2230,0,regular,NaN,NaN,NaN,NaN,...,Vaginal,0.07,1.0,0.0,magic.0010,Knights_Project_076_Pool3,magic.0010.S167.001.fa,PowerSoil,7251673,2016-12-10
magic.0004,101,no,12/15/16,NaN,1,regular,NaN,NaN,NaN,NaN,...,Vaginal,0.23,1.0,0.0,magic.0004,Knights_Project_046,magic.0004.S34.001.fa,MagAttract,1167038,2016-12-15
magic.0519,101,no,6/11/17,NaN,24,regular,NaN,NaN,NaN,NaN,...,Vaginal,6.09,1.0,0.0,magic.0519,Knights_Project_076_Pool3,magic.0519.S220.001.fa,PowerSoil,5725244,2017-06-11
magic.0859,101,no,9/10/17,1630,36,regular,NaN,NaN,NaN,formed,...,Vaginal,9.08,1.0,0.0,magic.0859,Knights_Project_068_Pool1,magic.0859.S197.001.fa,PowerSoil,9289438,2017-09-10
magic.1142,101,no,12/11/17,1330,48,regular,NaN,NaN,NaN,formed,...,Vaginal,12.11,0.0,0.0,magic.1142,Knights_Project_076_Pool1,magic.1142.S49.001.fa,PowerSoil,6666588,2017-12-11
magic.1388,101,no,2/14/18,800,NaN,Abx,NaN,during,5,formed,...,Vaginal,14.24,0.0,0.0,magic.1388,Knights_Project_068_Pool2,magic.1388.S398.001.fa,PowerSoil,8328134,2018-02-14
magic.1574,101,no,2/28/18,730,NaN,Abx,NaN,after,10,formed,...,Vaginal,14.70,0.0,0.0,magic.1574,Knights_Project_068_Pool2,magic.1574.S498.001.fa,PowerSoil,8190565,2018-02-28
magic.1781,101,no,4/9/18,NaN,60,regular,NaN,NaN,NaN,hard,...,Vaginal,16.02,0.0,0.0,magic.1781,Knights_Project_068_Pool4,magic.1781.S936.001.fa,PowerSoil,8728277,2018-04-09


KeyboardInterrupt: Interrupted by user

In [5]:
richness = pd.read_csv(os.path.join(config.CLEAN_DIR, "rarified_gene_richness.tsv"), sep="\t", index_col=0)
richness

,rarified_gene_richness
magic.0001,64331.0
magic.0004,69105.0
magic.0006,21372.0
magic.0007,89717.0
magic.0008,61474.0
...,...
magic.6595,185364.0
magic.6597,176493.0
magic.6598,142285.0
magic.6599,155385.0


In [6]:
assert(all(metadata.index == richness.index))
metadata["gene_richness"] = richness
metadata

,Patient_No,Hospital,Date_Collected,Time_Collected,Timeline_Weeks,Abx_regular,Abx_anomoly,Abx_timeline.prior..during..after.,abx_day,Consistency,...,calc_age_mo,currentfeed_bf,currentfeed_f,currentfeed_o,Sequencing_Project,Filename,Extraction,depth,date_time,gene_richness
Sample_ID,,,,,,,,,,,,,,,,,,,,,
magic.0001,101,no,12/10/16,1430,0,regular,NaN,NaN,NaN,NaN,...,0.07,1.0,0.0,magic.0001,Knights_Project_046,magic.0001.S22.001.fa,MagAttract,1544781,2016-12-10,64331.0
magic.0004,101,no,12/15/16,NaN,1,regular,NaN,NaN,NaN,NaN,...,0.23,1.0,0.0,magic.0004,Knights_Project_046,magic.0004.S34.001.fa,MagAttract,1167038,2016-12-15,69105.0
magic.0006,104,yes,12/15/16,2335,0,regular,NaN,NaN,NaN,NaN,...,0.03,1.0,1.0,magic.0006,Knights_Project_076_Pool3,magic.0006.S145.001.fa,PowerSoil,1720636,2016-12-15,21372.0
magic.0007,105,yes,12/15/16,2115,0,regular,NaN,NaN,NaN,NaN,...,0.03,1.0,1.0,magic.0007,Knights_Project_076_Pool3,magic.0007.S155.001.fa,PowerSoil,6825017,2016-12-15,89717.0
magic.0008,105,yes,12/15/16,1525,0,regular,NaN,NaN,NaN,NaN,...,0.03,1.0,1.0,magic.0008,Knights_Project_055,X8.S125.001.fa,PowerSoil,2928044,2016-12-15,61474.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
magic.6595,588,NaN,8/22/21,1702,96,regular,NaN,NaN,NaN,NaN,...,24.11,NaN,NaN,magic.6595,Knights_Project_077_Pool3,magic.6595.S253.001.fa,PowerSoilPro,1562968,2021-08-22,185364.0
magic.6597,595,NaN,9/6/21,NaN,96,regular,NaN,NaN,NaN,formed,...,24.11,NaN,NaN,magic.6597,Knights_Project_077_Pool3,magic.6597.S265.001.fa,PowerSoilPro,2113880,2021-09-06,176493.0
magic.6598,585,NaN,8/23/21,725,96,regular,NaN,NaN,NaN,formed,...,24.34,NaN,NaN,magic.6598,Knights_Project_077_Pool3,magic.6598.S277.001.fa,PowerSoilPro,1650388,2021-08-23,142285.0


In [7]:
metadata["abx_day"].value_counts()

10         232
2          193
5          186
30         135
0           83
na          26
1           16
6            6
7            6
3            5
4            3
8            2
13           2
14           2
9            2
20           2
31           1
11           1
32           1
5 or 10      1
2 & 5        1
Na           1
Name: abx_day, dtype: int64

In [8]:
metadata["Timeline_Weeks"].value_counts()

4     307
1     301
24    298
12    278
48    274
96    220
3     184
2     169
0     162
72    149
36    112
60     73
na     50
84     41
Name: Timeline_Weeks, dtype: int64

# subject gene richness z score for subject age vs abx state

In [9]:
from scipy.stats import zscore

zscores_list = []

for week in metadata["Timeline_Weeks"].unique():
    if week == "na": continue
    gene_richness = metadata[metadata["Timeline_Weeks"] == week]["gene_richness"]
    zscores = zscore(gene_richness)
    zscores_list.append(pd.DataFrame(zscores))

In [10]:
zscore_df = pd.concat(zscores_list, axis=0).iloc[:, [0]]
zscore_df.columns = ["zscore"]
zscore_df

,zscore
magic.0001,0.500181
magic.0006,-0.839210
magic.0007,1.291674
magic.0008,0.411104
magic.0010,0.432773
...,...
magic.6595,1.304466
magic.6597,1.069306
magic.6598,0.162491
magic.6599,0.509757


In [11]:
zscore_df

,zscore
magic.0001,0.500181
magic.0006,-0.839210
magic.0007,1.291674
magic.0008,0.411104
magic.0010,0.432773
...,...
magic.6595,1.304466
magic.6597,1.069306
magic.6598,0.162491
magic.6599,0.509757


In [12]:
metadata[metadata["Abx_regular"] == "Abx"]["gene_richness"].mean()

100589.5172413793

In [13]:
for subject in metadata["Patient_No"].unique():
    display(metadata[metadata["Patient_No"] == subject])
    input()
    

,Patient_No,Hospital,Date_Collected,Time_Collected,Timeline_Weeks,Abx_regular,Abx_anomoly,Abx_timeline.prior..during..after.,abx_day,Consistency,...,calc_age_mo,currentfeed_bf,currentfeed_f,currentfeed_o,Sequencing_Project,Filename,Extraction,depth,date_time,gene_richness
Sample_ID,,,,,,,,,,,,,,,,,,,,,
magic.0001,101,no,12/10/16,1430,0,regular,NaN,NaN,NaN,NaN,...,0.07,1.0,0.0,magic.0001,Knights_Project_046,magic.0001.S22.001.fa,MagAttract,1544781,2016-12-10,64331.0
magic.0004,101,no,12/15/16,NaN,1,regular,NaN,NaN,NaN,NaN,...,0.23,1.0,0.0,magic.0004,Knights_Project_046,magic.0004.S34.001.fa,MagAttract,1167038,2016-12-15,69105.0
magic.0010,101,no,12/10/16,2230,0,regular,NaN,NaN,NaN,NaN,...,0.07,1.0,0.0,magic.0010,Knights_Project_076_Pool3,magic.0010.S167.001.fa,PowerSoil,7251673,2016-12-10,62169.0
magic.0016,101,no,NaN,NaN,2,regular,NaN,NaN,NaN,NaN,...,NaN,1.0,0.0,magic.0016,Knights_Project_046,magic.0016.S82.001.fa,MagAttract,979090,NaT,65804.0
magic.0022,101,no,NaN,NaN,3,regular,NaN,NaN,NaN,NaN,...,NaN,1.0,0.0,magic.0022,Knights_Project_076_Pool3,magic.0022.S198.001.fa,PowerSoil,23427901,NaT,83135.0
magic.0042,101,no,NaN,NaN,4,regular,NaN,NaN,NaN,NaN,...,NaN,1.0,0.0,magic.0042,Knights_Project_068_Pool1,magic.0042.S60.001.fa,PowerSoil,8397705,NaT,95631.0
magic.0519,101,no,6/11/17,NaN,24,regular,NaN,NaN,NaN,NaN,...,6.09,1.0,0.0,magic.0519,Knights_Project_076_Pool3,magic.0519.S220.001.fa,PowerSoil,5725244,2017-06-11,88680.0
magic.0859,101,no,9/10/17,1630,36,regular,NaN,NaN,NaN,formed,...,9.08,1.0,0.0,magic.0859,Knights_Project_068_Pool1,magic.0859.S197.001.fa,PowerSoil,9289438,2017-09-10,111584.0
magic.1142,101,no,12/11/17,1330,48,regular,NaN,NaN,NaN,formed,...,12.11,0.0,0.0,magic.1142,Knights_Project_076_Pool1,magic.1142.S49.001.fa,PowerSoil,6666588,2017-12-11,104975.0


,Patient_No,Hospital,Date_Collected,Time_Collected,Timeline_Weeks,Abx_regular,Abx_anomoly,Abx_timeline.prior..during..after.,abx_day,Consistency,...,calc_age_mo,currentfeed_bf,currentfeed_f,currentfeed_o,Sequencing_Project,Filename,Extraction,depth,date_time,gene_richness
Sample_ID,,,,,,,,,,,,,,,,,,,,,
magic.0006,104,yes,12/15/16,2335,0,regular,NaN,NaN,NaN,NaN,...,0.03,1.0,1.0,magic.0006,Knights_Project_076_Pool3,magic.0006.S145.001.fa,PowerSoil,1720636,2016-12-15,21372.0
magic.0018,104,no,12/28/16,900,2,regular,NaN,NaN,NaN,watery,...,0.46,1.0,1.0,magic.0018,Knights_Project_046,magic.0018.S106.001.fa,MagAttract,2227237,2016-12-28,61271.0
magic.0020,104,no,12/21/16,1957,1,regular,NaN,NaN,NaN,watery,...,0.23,1.0,1.0,magic.0020,Knights_Project_046,magic.0020.S23.001.fa,MagAttract,1105790,2016-12-21,41560.0
magic.0048,104,no,1/4/17,1344,3,regular,NaN,NaN,NaN,watery,...,0.69,1.0,1.0,magic.0048,Knights_Project_046,magic.0048.S83.001.fa,MagAttract,1068893,2017-01-04,53128.0
magic.0078,104,no,1/11/17,2148,4,regular,NaN,NaN,NaN,watery,...,0.92,1.0,1.0,magic.0078,Knights_Project_046,magic.0078.S72.001.fa,MagAttract,1040799,2017-01-11,52497.0
magic.0231,104,no,3/14/17,1500,12,regular,NaN,NaN,NaN,watery,...,2.96,1.0,1.0,magic.0231,Knights_Project_046,magic.0231.S63.001.fa,MagAttract,1210079,2017-03-14,60048.0
magic.0527,104,no,6/14/17,1516,24,regular,NaN,NaN,NaN,watery,...,5.99,1.0,1.0,magic.0527,Knights_Project_046,magic.0527.S68.001.fa,MagAttract,1664226,2017-06-14,65777.0
magic.0894,104,no,9/19/17,1017,36,regular,NaN,NaN,NaN,formed,...,9.18,1.0,0.0,magic.0894,Knights_Project_055,X894.S241.001.fa,PowerSoil,2213238,2017-09-19,91067.0
magic.1115,104,no,12/14/17,855,48,regular,NaN,NaN,NaN,formed,...,12.01,1.0,0.0,magic.1115,Knights_Project_055,X1115.S329.001.fa,PowerSoil,2790906,2017-12-14,89893.0


,Patient_No,Hospital,Date_Collected,Time_Collected,Timeline_Weeks,Abx_regular,Abx_anomoly,Abx_timeline.prior..during..after.,abx_day,Consistency,...,calc_age_mo,currentfeed_bf,currentfeed_f,currentfeed_o,Sequencing_Project,Filename,Extraction,depth,date_time,gene_richness
Sample_ID,,,,,,,,,,,,,,,,,,,,,
magic.0007,105,yes,12/15/16,2115,0,regular,NaN,NaN,NaN,NaN,...,0.03,1.0,1.0,magic.0007,Knights_Project_076_Pool3,magic.0007.S155.001.fa,PowerSoil,6825017,2016-12-15,89717.0
magic.0008,105,yes,12/15/16,1525,0,regular,NaN,NaN,NaN,NaN,...,0.03,1.0,1.0,magic.0008,Knights_Project_055,X8.S125.001.fa,PowerSoil,2928044,2016-12-15,61474.0
magic.0012,105,no,12/22/16,950,1,regular,NaN,NaN,NaN,soft,...,0.26,1.0,1.0,magic.0012,Knights_Project_076_Pool3,magic.0012.S177.001.fa,PowerSoil,8106772,2016-12-22,95076.0
magic.0017,105,no,12/28/16,1450,2,regular,NaN,NaN,NaN,watery,...,0.46,1.0,1.0,magic.0017,Knights_Project_046,magic.0017.S94.001.fa,MagAttract,1578676,2016-12-28,82929.0
magic.0044,105,no,1/4/17,2137,3,regular,NaN,NaN,NaN,watery,...,0.69,1.0,1.0,magic.0044,Knights_Project_076_Pool3,magic.0044.S147.001.fa,PowerSoil,7582355,2017-01-04,94073.0
magic.0064,105,no,1/11/17,2125,4,regular,NaN,NaN,NaN,watery,...,0.92,1.0,1.0,magic.0064,Knights_Project_055,X64.S126.001.fa,PowerSoil,3123543,2017-01-11,68228.0
magic.0228,105,no,3/15/17,1700,12,regular,NaN,NaN,NaN,watery,...,2.99,1.0,1.0,magic.0228,Knights_Project_055,X228.S136.001.fa,PowerSoil,2965203,2017-03-15,91839.0
magic.0541,105,no,6/14/17,1330,24,regular,NaN,NaN,NaN,formed,...,5.99,1.0,1.0,magic.0541,Knights_Project_046,magic.0541.S45.001.fa,MagAttract,1268539,2017-06-14,54442.0
magic.0880,105,no,9/15/17,800,36,regular,NaN,NaN,NaN,soft,...,9.05,0.0,1.0,magic.0880,Knights_Project_055,X880.S187.001.fa,PowerSoil,2377697,2017-09-15,61627.0


,Patient_No,Hospital,Date_Collected,Time_Collected,Timeline_Weeks,Abx_regular,Abx_anomoly,Abx_timeline.prior..during..after.,abx_day,Consistency,...,calc_age_mo,currentfeed_bf,currentfeed_f,currentfeed_o,Sequencing_Project,Filename,Extraction,depth,date_time,gene_richness
Sample_ID,,,,,,,,,,,,,,,,,,,,,
magic.0011,107,no,12/25/16,1731,1,regular,NaN,NaN,NaN,watery,...,0.23,1.0,0.0,magic.0011,Knights_Project_046,magic.0011.S46.001.fa,MagAttract,1305578,2016-12-25,22788.0
magic.0024,107,yes,12/19/16,1705,0,regular,NaN,NaN,NaN,NaN,...,0.03,1.0,0.0,magic.0024,Knights_Project_055,X24.S133.001.fa,PowerSoil,3691700,2016-12-19,19476.0
magic.0025,107,yes,12/20/16,407,0,regular,NaN,NaN,NaN,NaN,...,0.07,1.0,0.0,magic.0025,Knights_Project_076_Pool3,magic.0025.S135.001.fa,PowerSoil,7428214,2016-12-20,14587.0
magic.0050,107,no,1/1/17,1754,2,regular,NaN,NaN,NaN,watery,...,0.46,1.0,0.0,magic.0050,Knights_Project_046,magic.0050.S95.001.fa,MagAttract,1105272,2017-01-01,31142.0
magic.0060,107,no,1/9/17,1430,3,regular,NaN,NaN,NaN,soft,...,0.72,1.0,0.0,magic.0060,Knights_Project_068_Pool1,magic.0060.S2.001.fa,PowerSoil,8633339,2017-01-09,75239.0
magic.0080,107,no,1/14/17,2130,4,regular,NaN,NaN,NaN,soft,...,0.89,1.0,0.0,magic.0080,Knights_Project_055,X80.S173.001.fa,PowerSoil,2336036,2017-01-14,40185.0
magic.0246,107,no,3/14/17,1145,12,regular,NaN,NaN,NaN,watery,...,2.83,1.0,0.0,magic.0246,Knights_Project_046,magic.0246.S28.001.fa,MagAttract,941861,2017-03-14,46316.0
magic.0538,107,no,6/17/17,1600,24,regular,NaN,NaN,NaN,soft,...,5.95,1.0,0.0,magic.0538,Knights_Project_046,magic.0538.S116.001.fa,MagAttract,1453847,2017-06-17,29129.0
magic.0893,107,no,9/18/17,1330,36,regular,NaN,NaN,NaN,formed,...,9.01,1.0,0.0,magic.0893,Knights_Project_055,X893.S229.001.fa,PowerSoil,2314616,2017-09-18,78471.0


KeyboardInterrupt: Interrupted by user

In [14]:
subjects = metadata["Patient_No"].unique()
subjects

array([101, 104, 105, 107, 102, 106, 109, 108, 119, 122, 117, 115, 120,
       111, 116, 125, 124, 126, 127, 136, 131, 137, 135, 142, 133, 140,
       144, 143, 141, 123, 149, 148, 151, 150, 147, 155, 153, 158, 162,
       160, 145, 161, 167, 166, 168, 169, 165, 170, 157, 164, 172, 174,
       173, 181, 185, 180, 175, 187, 183, 177, 184, 186, 188, 189, 193,
       198, 195, 132, 192, 190, 191, 196, 197, 199, 138, 202, 204, 203,
       208, 201, 205, 209, 211, 210, 217, 219, 218, 222, 224, 220, 221,
       223, 225, 213, 227, 233, 229, 231, 226, 235, 228, 247, 237, 238,
       239, 245, 243, 241, 249, 242, 253, 250, 251, 263, 256, 254, 258,
       257, 266, 260, 261, 267, 264, 248, 265, 268, 259, 270, 252, 271,
       269, 275, 276, 272, 280, 277, 282, 281, 234, 273, 283, 285, 289,
       286, 287, 290, 293, 298, 299, 297, 300, 302, 301, 295, 308, 307,
       312, 304, 305, 274, 313, 318, 296, 306, 315, 322, 319, 324, 333,
       328, 321, 329, 330, 332, 323, 341, 335, 327, 340, 337, 33

In [15]:
metadata

,Patient_No,Hospital,Date_Collected,Time_Collected,Timeline_Weeks,Abx_regular,Abx_anomoly,Abx_timeline.prior..during..after.,abx_day,Consistency,...,calc_age_mo,currentfeed_bf,currentfeed_f,currentfeed_o,Sequencing_Project,Filename,Extraction,depth,date_time,gene_richness
Sample_ID,,,,,,,,,,,,,,,,,,,,,
magic.0001,101,no,12/10/16,1430,0,regular,NaN,NaN,NaN,NaN,...,0.07,1.0,0.0,magic.0001,Knights_Project_046,magic.0001.S22.001.fa,MagAttract,1544781,2016-12-10,64331.0
magic.0004,101,no,12/15/16,NaN,1,regular,NaN,NaN,NaN,NaN,...,0.23,1.0,0.0,magic.0004,Knights_Project_046,magic.0004.S34.001.fa,MagAttract,1167038,2016-12-15,69105.0
magic.0006,104,yes,12/15/16,2335,0,regular,NaN,NaN,NaN,NaN,...,0.03,1.0,1.0,magic.0006,Knights_Project_076_Pool3,magic.0006.S145.001.fa,PowerSoil,1720636,2016-12-15,21372.0
magic.0007,105,yes,12/15/16,2115,0,regular,NaN,NaN,NaN,NaN,...,0.03,1.0,1.0,magic.0007,Knights_Project_076_Pool3,magic.0007.S155.001.fa,PowerSoil,6825017,2016-12-15,89717.0
magic.0008,105,yes,12/15/16,1525,0,regular,NaN,NaN,NaN,NaN,...,0.03,1.0,1.0,magic.0008,Knights_Project_055,X8.S125.001.fa,PowerSoil,2928044,2016-12-15,61474.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
magic.6595,588,NaN,8/22/21,1702,96,regular,NaN,NaN,NaN,NaN,...,24.11,NaN,NaN,magic.6595,Knights_Project_077_Pool3,magic.6595.S253.001.fa,PowerSoilPro,1562968,2021-08-22,185364.0
magic.6597,595,NaN,9/6/21,NaN,96,regular,NaN,NaN,NaN,formed,...,24.11,NaN,NaN,magic.6597,Knights_Project_077_Pool3,magic.6597.S265.001.fa,PowerSoilPro,2113880,2021-09-06,176493.0
magic.6598,585,NaN,8/23/21,725,96,regular,NaN,NaN,NaN,formed,...,24.34,NaN,NaN,magic.6598,Knights_Project_077_Pool3,magic.6598.S277.001.fa,PowerSoilPro,1650388,2021-08-23,142285.0
